In [1]:
import numpy as np
import matplotlib.pyplot as plt

import sys
sys.path.append("../src/qspright/")
sys.path.append("src")
sys.path.append("src/rna_transform")

import rna_transform.utils as utils
import rna_transform.rna_utils as rna_utils

from qspright.utils import dec_to_qary_vec

In [2]:
q = 4
positions = rna_utils.RNA_POSITIONS

# positions += [3,4]
# rna_utils.RNA_POSITIONS = positions

n = len(positions)

print("n = {}, N = {}".format(n, n ** q))

n = 10, N = 10000


### get Lasso coefficients

In [3]:
# emp_beta_lasso = rna_utils.calculate_rna_lasso(save=True)

In [4]:
# Calculate fraction of variance explained by empirical coefficients
# bm_fv_lasso = utils.calc_frac_var_explained(emp_beta_lasso)

### get GWHT coefficients

In [5]:
emp_beta_gwht = rna_utils.calculate_rna_gwht(save=True)
print(emp_beta_gwht)

Calculating free energies...


  1%|▏         | 13328/1048576 [00:35<46:28, 371.24it/s]


KeyboardInterrupt: 

In [ ]:
# Calculate fraction of variance explained by empirical coefficients
bm_fv_gwht = utils.calc_frac_var_explained(emp_beta_gwht)

### get GWHT coefficients with QSPRIGHT

In [ ]:
num_subsample = 2
num_random_delays = 5
b = 4
gwht, (n_used, n_used_unique), peeled = rna_utils.calculate_rna_qspright(save=False, report = True, num_subsample = num_subsample, num_random_delays = num_random_delays, b = b)
emp_beta_qspright = np.reshape(gwht, -1)
print("total sample ratio = ", n_used / q ** n)
print("unique sample ratio = ", n_used_unique / q ** n)
print("found sparsity = ", np.sum(np.abs(emp_beta_qspright) > 1e-5))
print("NMSE = ", np.sum(np.abs(emp_beta_gwht - emp_beta_qspright)**2) / np.sum(np.abs(emp_beta_gwht)**2))

In [ ]:
# Calculate fraction of variance explained by empirical coefficients
bm_fv_qspright = utils.calc_frac_var_explained(emp_beta_qspright)

In [ ]:
# plt.plot(bm_fv_lasso[:300], label = "Fourier")
plt.plot(bm_fv_gwht[:600], label = "GWHT")
plt.plot(bm_fv_qspright[:600], label = "QSPRIGHT")
plt.legend()
plt.show()

In [ ]:
b_list = [4, 5, 6]
num_subsample_list = [2, 3, 4]
num_random_delays_list = [3, 5, 7]
iters = 1

sample_ratios = np.zeros((iters, len(b_list), len(num_subsample_list), len(num_random_delays_list)))
unique_sample_ratios = np.zeros((iters, len(b_list), len(num_subsample_list), len(num_random_delays_list)))
nmse = np.zeros((iters, len(b_list), len(num_subsample_list), len(num_random_delays_list)))

for idx in range(iters):
    print(idx)
    for s, num_subsample in enumerate(num_subsample_list):
        for d, num_random_delays in enumerate(num_random_delays_list):
            for b_idx, b in enumerate(b_list):
                gwht, (n_used, n_used_unique), peeled = rna_utils.calculate_rna_qspright(save=False, report = True, num_subsample = num_subsample, num_random_delays = num_random_delays, b = b)
                emp_beta_qspright = np.reshape(gwht, -1)
                sample_ratios[idx, b_idx, s, d] = n_used/ q ** n
                unique_sample_ratios[idx, b_idx, s, d] = n_used_unique/ q ** n
                nmse[idx, b_idx, s, d] = np.sum(np.abs(emp_beta_gwht - emp_beta_qspright)**2) / np.sum(np.abs(emp_beta_gwht)**2)

In [ ]:
for i, b in enumerate(b_list):
    s_values = np.mean(unique_sample_ratios[:, i], axis=0).flatten()
    mse_values =  np.mean(nmse[:, i], axis=0).flatten()

    plt.scatter(s_values, mse_values, label = "b = {}".format(b))

plt.xlabel('Unique Sample Ratio')
plt.ylabel('NMSE')
plt.legend()
plt.show()

In [ ]:
for i, b in enumerate(b_list):
    s_values = np.mean(sample_ratios[:, i], axis=0).flatten()
    mse_values =  np.mean(nmse[:, i], axis=0).flatten()

    plt.scatter(s_values, mse_values, label = "b = {}".format(b))

plt.xlabel('Total Sample Ratio')
plt.ylabel('NMSE')
plt.legend()
plt.show()

### compare indices with most energy

In [ ]:
beta_power = np.abs(emp_beta_gwht) ** 2
for _ in range(30):
    ind = beta_power.argmax()
    print(beta_power[ind], np.array(dec_to_qary_vec([ind], n = n, q=q))[:, 0])
    beta_power[ind] = 0

In [ ]:
beta_power = np.abs(emp_beta_qspright) ** 2
for _ in range(30):
    ind = beta_power.argmax()
    print(beta_power[ind], np.array(dec_to_qary_vec([ind], n = n, q=q))[:, 0])
    beta_power[ind] = 0